In [53]:
# !export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/lib64
!env CUDA_DEVICE_ORDER=PCI_BUS_ID
!env CUDA_VISIBLE_DEVICES=4

LMOD_DIR=/usr/share/lmod/lmod/libexec/
TMUX=/tmp/tmux-22086/default,1030980,0
SSH_CLIENT=103.107.4.31 63325 22
USER=m13518040
LMOD_COLORIZE=yes
MPLBACKEND=module://matplotlib_inline.backend_inline
LMOD_PKG=/usr/share/lmod/lmod
LD_LIBRARY_PATH=/usr/local/cuda/lib:/usr/local/cuda/lib
HOME=/home/m13518040
MOTD_SHOWN=pam
OLDPWD=/raid/data/m13518040
SSH_TTY=/dev/pts/2
PAGER=cat
TF_CPP_MIN_LOG_LEVEL=1
LMOD_sys=Linux
TF2_BEHAVIOR=1
LOGNAME=m13518040
TERM=xterm-color
CUDA_DEVICE_ORDER=PCI_BUS_ID
PATH=/home/m13518040/.local/bin:/usr/local/cuda/bin:/opt/bin/:/usr/local/cuda/bin:/home/m13518040/.local/bin:/usr/local/cuda/bin:/opt/bin/:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin
LMOD_FULL_SETTARG_SUPPORT=no
LMOD_PREPEND_BLOCK=normal
KMP_DUPLICATE_LIB_OK=True
KMP_INIT_AT_FORK=FALSE
LANG=en_US.UTF-8
MODULEPATH_ROOT=/sw/modules
SHELL=/bin/sh
LMOD_CMD=/usr/share/lmod/lmod/libexec/lmod
BASH_ENV=/usr/share/lmod/lmod/init/bash
GIT

In [54]:
from utils.utils_init_model import text_logit, fine_tuning_model, eval_model, init_model, load_word_index
import time
from nltk.tokenize import word_tokenize
import torch
import torch.nn.functional as F
# import jax.numpy as jnp
# from jax import random

import math
# from .attack_helper import get_synonyms, codemix_perturbation, synonym_replacement, swap_minimum_importance_words, trans_dict

import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words_set = []
for w in stopwords.words('indonesian'):
    stop_words_set.append(w)

from icecream import ic


import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
# nltk.download('omw-1.4')

import copy
from deep_translator import GoogleTranslator
import numpy as np
import random
# from utils.utils_semantic_use import USE
from utils.utils_init_dataset import set_seed, load_dataset_loader
from sklearn.metrics import accuracy_score

import torch
device = torch.device("cuda")

import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import tensorflow_hub as hub
import os

import jax.numpy as jnp

# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = "7"
# tf.config.run_functions_eagerly(True)

In [55]:
# !pip install jax
# !pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_releases.html  # Note: wheels only available on linux.

In [56]:
class USE(object):
    def __init__(self):
        import tensorflow_hub as hub
        super(USE, self).__init__()
        module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
        self.embed = hub.load(module_url)

    def semantic_sim(self, sentA : str, sentB : str) -> float:
        """
        Args:
            sentA: The first sentence.
            sentB: The second sentence.

        Returns:
            Cosine distance between two sentences.
        
        """
        if sentA.lower() == sentB.lower():
            return 1.000
        ret = np.array(self.embed([sentA, sentB]).numpy())
        score = ret[0].dot(ret[1]) / (np.linalg.norm(ret[0]) * np.linalg.norm(ret[1]))
        if score>1:
            return 1.000
        return score

    def after_attack(self, input, adversarial_sample):
        if adversarial_sample is not None:
            return self.calc_score(input["x"], adversarial_sample)
# tf.config.run_functions_eagerly(True)
# USE()

In [57]:
# class USE(object):
#     def __init__(self, module_url=None):
#         super(USE, self).__init__()
#         module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
#         self.embed = hub.load(module_url)
        
#         def embed(input):
#             return model(input)
        
#         # config = tf.compat.v1.ConfigProto()
#         # config.gpu_options.allow_growth = True
#         # self.sess = tf.compat.v1.Session(config=config)
#         # self.sess.run([tf.compat.v1.global_variables_initializer(), tf.compat.v1.tables_initializer()])

#     def semantic_sim(self, sents1, sents2):
#         # with tf.Session() as session:
#         #     session.run([tf.global_variables_initializer(), tf.tables_initializer()])
#         #     with tf.device('GPU'):
#         #         message_embeddings_ = session.run(self.embed([sents1, sents2]))
        
#         if sents1.lower() == sents2.lower():
#             return 1.000
        
#         message_embeddings_ = self.embed([sents1, sents2])
#         # message_embeddings_ = message_embeddings_.eval(session=self.sess)
#         corr = np.tensordot(message_embeddings_, message_embeddings_, axes=(-1,-1))
        
#         if corr[0][1] > 1:
#             return 1.000
#         return corr[0][1]

In [58]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

def get_synonyms(word):
    if word in stop_words_set:
        return [word]

    word_lemmas = wordnet.lemmas(word, lang="ind")
    
    hypernyms = []
    for lem in word_lemmas:
        hypernyms.append(lem.synset().hypernyms())

    if not any(hypernyms):
        return [word]
    
    lemma_corp = []
    
    for hypernym in hypernyms:
        if(len(hypernym) < 1):
            continue
        else:
            lemma_corp.append(hypernym[0].lemmas(lang="ind"))
            
    lemmas = set()
    for list_lemmas in lemma_corp:
        if(len(list_lemmas) < 1):
            lemmas.add(word)
        else:
            for l in list_lemmas:
                lemmas.add(l.name())
    
    clean_synonyms = set()
    for syn in lemmas.copy():
        synonym = syn.replace("_", " ").replace("-", " ").lower()
        synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
        clean_synonyms.add(synonym) 
    if word in clean_synonyms:
        clean_synonyms.remove(word)
    
    if len(list(clean_synonyms)) < 1:
        return [word]
    else:
        return list(clean_synonyms)

def codemix_perturbation(words, target_lang, words_perturb):
    """
    'su': 'sundanese'
    'jw': 'javanese'
    'ms': 'malay'
    'en': 'english'
    """
    translator = GoogleTranslator(source="id", target=target_lang)
    new_wp = []
    for wp in words_perturb:
        if wp[1].isalpha():
            new_wp.append(wp[1])
    
    if len(new_wp) == 1:
        new_wp_trans = dict(zip(new_wp, translator.translate(new_wp[0])))
    elif len(new_wp) == 0:
        return ' '.join(new_words), {}
        
    new_wp_trans = dict(zip(new_wp, translator.translate_batch(new_wp)))
    
    supported_langs = ["su", "jw", "ms", "en", "fr", "it"]
    
    if target_lang not in supported_langs:
        raise ValueError('Language Unavailable')
    
    new_words = words.copy()
    
    if len(words_perturb) >= 1:
        for perturb_word in new_wp_trans.keys():
            new_words = [new_wp_trans[perturb_word] if word == perturb_word and word.isalpha() else word for word in new_words]

    return ' '.join(new_words), new_wp_trans

def synonym_replacement(words, words_perturb):    
    # new_words = words.copy()
    new_wp = []
    for wp in words_perturb:
        if wp[1].isalpha():
            new_wp.append(wp[1])
    
    if len(new_wp) == 1:
        new_wp_trans = dict(zip(new_wp, get_synonyms(new_wp[0])[0]))
    elif len(new_wp) == 0:
        return ' '.join(new_words), {}
    
    wp_trans=[]
    for wp in new_wp:
        wp_trans.append(get_synonyms(wp)[0])
    
    new_wp_trans = dict(zip(new_wp, wp_trans))
    
    new_words = words.copy()
    
    if len(words_perturb) >= 1:
        for perturb_word in new_wp_trans.keys():
            new_words = [new_wp_trans[perturb_word[1]] if word == perturb_word[1] and word.isalpha() else word for word in new_words]

    sentence = ' '.join(new_words)
    
    return sentence, new_wp_trans

# fungsi untuk mencari kandidat lain ketika sebuah kandidat perturbasi kurang dari sim_score_threshold
def swap_minimum_importance_words(words_perturb, top_importance_words):
    print("pointer")
    def get_minimums(word_tups):
        arr = []
        for wt in word_tups:
            if wt[2] == min(top_importance_words, key = lambda t: t[2])[2]:
                arr.append(wt)
        return arr
    
    # get list of words with minimum importance score
    minimum_import = get_minimums(top_importance_words)
    unlisted = list(set(words_perturb).symmetric_difference(set(top_importance_words)))
    
    ic(unlisted)
    
    len_wp = len(top_importance_words)
    len_ul = len(unlisted)
    
    res = []
    for i in range(len_wp):
        if top_importance_words[i] in minimum_import:
            temp_wp = list(copy.deepcopy(top_importance_words))
            if len(temp_wp) == 0:
                break
            
            swapped_wp = np.array([(temp_wp) for i in range(len_ul)])
            
            ic(swapped_wp)
            for j in range(len(swapped_wp)):
                temp_sm = np.vstack((swapped_wp[j], tuple(unlisted[j])))
                
                res.append(temp_sm.tolist())
            temp_wp.pop(i)
                
    return res

def trans_dict(wordlist, perturbation_technique, lang=None):
    translated_wordlist = {}
    # ic(wordlist)
    if perturbation_technique == "codemixing":
        translator = GoogleTranslator(source="id", target=lang)
        
        for word in wordlist:
            if word.isalpha():
                translated_wordlist[word] = translator.translate(word)
            else:
                translated_wordlist[word] = word
    else:
        for word in wordlist:
            translated_wordlist[word] = get_synonyms(word)[0]
            
    return translated_wordlist

In [69]:
def logit_prob(text_ls, predictor, tokenizer):
    with torch.inference_mode():
        subwords = np.array(tokenizer(text_ls)['input_ids'])
        ic(text_ls)
        ic(subwords)
        ic(subwords.ndim)
        # ic(subwords)
        if subwords.ndim <= 1:
            subwords = torch.from_numpy(subwords).view(1, -1).to(predictor.device)
            logits = predictor(subwords)[0]
            orig_label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
            orig_probs = F.softmax(logits, dim=-1).squeeze()
            orig_prob = F.softmax(logits, dim=-1).squeeze()[orig_label].detach().cpu().numpy()
        else:
            subwords = torch.from_numpy(subwords).to(predictor.device)
            logits = finetuned_model(subwords)[0]
            orig_label = [t.squeeze().item() for t in torch.topk(logits, k=1, dim=-1)[1]]
            orig_probs = F.softmax(logits, dim=-1).squeeze()
            orig_prob = [F.softmax(logits, dim=-1).squeeze()[idx][label].detach().cpu().numpy() for idx, label in enumerate(orig_label)]
    
    return orig_label, orig_probs, orig_prob

def attack(text_ls,
           true_label,
           predictor,
           tokenizer,
           att_ratio,
           perturbation_technique,
           lang_codemix=None,
           sim_predictor=None,
           sim_score_threshold=0,
           sim_score_window=15,
           batch_size=32, 
           import_score_threshold=-1.):
    
    start_time = time.time()
    
    label_dict = {
        'positive': 0, 
        'neutral': 1, 
        'negative': 2}
    
    original_text = text_ls
    orig_label, orig_probs, orig_prob = logit_prob(text_ls, predictor, tokenizer)
    
    if len(original_text.split()) < sim_score_window:
        sim_score_threshold = 0.1  
    
#     SEK SALAAHHHHHH
    if true_label != orig_label:
        running_time = round(time.time() - start_time, 2)
        # perturbed_text, perturbed_semantic_sim, orig_label, orig_probs, perturbed_label, perturbed_probs, translated_words, running_time
        orig_probs = np.round(orig_probs.detach().cpu().numpy().tolist(),4)
        return original_text, 1.000, orig_label, orig_probs, orig_label, orig_probs,'', running_time
    else:
        text_ls = original_text.split()
        text_ls = [word for word in text_ls if word.isalnum()]
        len_text = len(text_ls)
        ic(len_text)
        half_sim_score_window = (sim_score_window - 1) // 2
        # num_queries = 1
        
        leave_1_texts = [' '.join(text_ls[:ii] + [tokenizer.mask_token] + text_ls[min(ii + 1, len_text):]) for ii in range(len_text)]
                
        leave_1_probs = []
        leave_1_probs_argmax = []
        # num_queries += len(leave_1_texts)
        
# torch.inference mode
        start_time_logit = time.time()
        leave_1_probs_argmax, leave_1_probs, _ = logit_prob(leave_1_texts, predictor, tokenizer)
        
        
        end_time_logit = round(time.time() - start_time_logit, 4)
        ic(end_time_logit)
        

        
        start_time_arrops = time.time()
        # leave_1_probs = torch.tensor(np.array(leave_1_probs)).to("cuda:0")
        leave_1_probs = torch.tensor(np.array(leave_1_probs.detach().cpu().numpy())).to("cuda:0")
        ic(leave_1_probs)
        
        orig_prob_extended=np.empty(len_text)
        orig_prob_extended.fill(orig_prob)
        ic(orig_prob_extended)
        orig_prob_extended = torch.tensor(orig_prob_extended).to("cuda:0")
        
        arr1 = orig_prob_extended - leave_1_probs[:,orig_label] + float(leave_1_probs_argmax != orig_label)
        arr2 = (leave_1_probs.max(dim=-1)[0].to("cuda:0") - orig_probs[leave_1_probs_argmax].to("cuda:0"))
        
        import_scores = arr1*arr2
        import_scores = [im * -1 for im in import_scores]
        
        words_perturb = []
        for idx, score in sorted(enumerate(import_scores), key=lambda x: x[1], reverse=True):
            try:
                if score > import_score_threshold and text_ls[idx] not in stop_words_set:
                    words_perturb.append((idx, text_ls[idx], score.item()))
            except Exception as e:
                print(e)
                print(idx, len(text_ls), import_scores.shape, text_ls, len(leave_1_texts))
        
        num_perturbation = math.floor(len(words_perturb)*att_ratio)
        
        end_time_arrops = round(time.time() - start_time_arrops, 4)
        ic(end_time_arrops)
        
        if num_perturbation < 1:
            num_perturbation = 1
        
#       top words perturb berisi list kata terpenting yang tidak akan diswitch ketika first_codemix_sim_score < sim_score_threshold
        top_words_perturb = words_perturb[:num_perturbation]
        trans_word = [twp[1] for twp in top_words_perturb]
        
        running_time_a = round(time.time() - start_time, 4)
        ic(running_time_a)
        
        start_time_pert = time.time()
        if perturbation_technique == "codemixing":
            perturbed_text,translated_words = codemix_perturbation(text_ls, lang_codemix, top_words_perturb)
        elif perturbation_technique == "synonym_replacement":
            perturbed_text,translated_words = synonym_replacement(text_ls, top_words_perturb)
        end_time_pert = round(time.time() - start_time_pert, 4)
        ic(end_time_pert)
        
        
        start_time_use = time.time()
        first_perturbation_sim_score = sim_predictor.semantic_sim(original_text, perturbed_text)
        end_time_use = round(time.time() - start_time_use, 4)
        
        ic(end_time_use)
        
        ic(first_perturbation_sim_score)
        
        words_perturb_candidates = []
        if first_perturbation_sim_score < sim_score_threshold:
            words_perturb_candidates.append(top_words_perturb)
            swapped = swap_minimum_importance_words(words_perturb, top_words_perturb)
            for s in swapped:
                words_perturb_candidates.append(s)

            words_perturb_candidates = [[tuple(w) for w in wpc] for wpc in words_perturb_candidates]

            candidate_comparison = {}
            for wpc in words_perturb_candidates:
                
                if perturbation_technique == "codemixing":
#                     bisa dicache, gaperlu request satu2
                    perturbed_candidate,translated_words = codemix_perturbation(text_ls, lang_codemix, wpc)
                elif perturbation_technique == "synonym_replacement":
                    perturbed_candidate,translated_words = synonym_replacement(text_ls, wpc)
                    
                perturbed_candidate_sim_score = sim_predictor.semantic_sim(original_text, perturbed_candidate)
                
                candidate_comparison[perturbed_candidate] = (perturbed_candidate_sim_score, wpc[-1][-1], wpc)

            sorted_candidate_comparison = sorted(candidate_comparison.keys(), key=lambda x: (float(candidate_comparison[x][0]), float(candidate_comparison[x][1])), reverse=True)

            perturbed_text = sorted_candidate_comparison[0]
            
            top_words_perturb = candidate_comparison[sorted_candidate_comparison[0]][-1]
            trans_word = [twp[1] for twp in top_words_perturb]
            
        perturbed_semantic_sim = sim_predictor.semantic_sim(original_text, perturbed_text)
        if perturbed_semantic_sim < sim_score_threshold:
            perturbed_text = original_text
            perturbed_semantic_sim = 1.000
        
        perturbed_label, perturbed_probs, perturbed_prob = logit_prob(perturbed_text, predictor, tokenizer)
        
        
        # translated_words = trans_dict(trans_word, perturbation_technique, lang_codemix)
        
        orig_probs = np.round(orig_probs.detach().cpu().numpy().tolist(),4)
        perturbed_probs = np.round(perturbed_probs.detach().cpu().numpy().tolist(),4)
        
        running_time = round(time.time() - start_time, 4)
        ic(running_time)
        
        # ic(perturbed_text, perturbed_semantic_sim, orig_label, orig_probs, perturbed_label, perturbed_probs, translated_words, running_time)
        return perturbed_text, perturbed_semantic_sim, orig_label, orig_probs, perturbed_label, perturbed_probs, translated_words, running_time

In [60]:
model_target="IndoBERT"
downstream_task="sentiment"
attack_strategy="adversarial"
# finetune_epoch=1
num_sample=50
# exp_name=
perturbation_technique="codemixing"
perturb_ratio=0.4
perturb_lang="en"
seed=26092020
dataset="valid"

set_seed(seed)
use = USE()

In [61]:
tokenizer, config, finetuned_model = init_model(model_target, downstream_task, seed)
w2i, i2w = load_word_index(downstream_task)

train_dataset, train_loader, train_path = load_dataset_loader(downstream_task, 'train', tokenizer)
valid_dataset, valid_loader, valid_path = load_dataset_loader(downstream_task, 'valid', tokenizer)
test_dataset, test_loader, test_path = load_dataset_loader(downstream_task, 'test', tokenizer)

finetuned_model.to(device)
# finetuned_model = fine_tuning_model(model, i2w, train_loader, valid_loader, finetune_epoch)

if dataset == "valid":
    exp_dataset = valid_dataset.load_dataset(valid_path)[380:430]
elif dataset == "train":
    exp_dataset = train_dataset.load_dataset(train_path)
# exp_dataset = dd.from_pandas(exp_dataset, npartitions=10)
# exp_dataset = te.DataFrame.from_pandas(exp_dataset)
text,label = None,None

In [70]:
if downstream_task == 'sentiment':
    text = 'text'
    label = 'sentiment'
    exp_dataset[['perturbed_text', 'perturbed_semantic_sim', 'pred_label', 'pred_proba', 'perturbed_label', 'perturbed_proba', 'translated_word(s)', 'running_time(s)']] = exp_dataset.progress_apply(
        lambda row: attack(
            text_ls = row.text,
            true_label = row.sentiment,
            predictor = finetuned_model,
            tokenizer = tokenizer, 
            att_ratio = perturb_ratio,
            perturbation_technique = perturbation_technique,
            lang_codemix = perturb_lang,
            sim_predictor = use), axis=1, result_type='expand'
    )
elif downstream_task == 'emotion':
    text = 'tweet'
    label = 'label'
    exp_dataset[['perturbed_text', 'perturbed_semantic_sim', 'pred_label', 'pred_proba', 'perturbed_label', 'perturbed_proba', 'translated_word(s)', 'running_time(s)']] = exp_dataset.progress_apply(
        lambda row: attack(
            text_ls = row.tweet.values,
            true_label = row.label.values,
            predictor = finetuned_model,
            tokenizer = tokenizer, 
            att_ratio = perturb_ratio,
            perturbation_technique = perturbation_technique,
            lang_codemix = perturb_lang,
            sim_predictor = use), axis=1, result_type='expand'
    )

before_attack = accuracy_score(exp_dataset[label], exp_dataset['pred_label'])
after_attack = accuracy_score(exp_dataset[label], exp_dataset['perturbed_label'])

exp_dataset.loc[exp_dataset.index[0], 'before_attack_acc'] = before_attack
exp_dataset.loc[exp_dataset.index[0], 'after_attack_acc'] = after_attack
exp_dataset.loc[exp_dataset.index[0], 'avg_semantic_sim'] = exp_dataset["perturbed_semantic_sim"].mean()
exp_dataset.loc[exp_dataset.index[0], 'avg_running_time(s)'] = exp_dataset["running_time(s)"].mean()

  0%|                                                                        | 0/50 [00:00<?, ?it/s]ic| text_ls: 'mau marah sama koneksi indosat'
ic| subwords: array([   2,  422, 4700,  500, 5622, 7884,    3])
ic| subwords.ndim: 1
ic| len_text: 5
ic| text_ls: ['[MASK] marah sama koneksi indosat',
              'mau [MASK] sama koneksi indosat',
              'mau marah [MASK] koneksi indosat',
              'mau marah sama [MASK] indosat',
              'mau marah sama koneksi [MASK]']
ic| subwords: array([[   2,    4, 4700,  500, 5622, 7884,    3],
                     [   2,  422,    4,  500, 5622, 7884,    3],
                     [   2,  422, 4700,    4, 5622, 7884,    3],
                     [   2,  422, 4700,  500,    4, 7884,    3],
                     [   2,  422, 4700,  500, 5622,    4,    3]])
ic| subwords.ndim: 2
ic| end_time_logit: 0.1402
ic| leave_1_probs: tensor([[5.8145e-04, 1.2208e-03, 9.9820e-01],
                           [3.1326e-03, 9.9333e-01, 3.5423e-03],
     

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
exp_dataset.head(10)

### After Logit Improvement

In [41]:
attack(
    text_ls = exp_dataset.iloc[[0]].text.item(),
    true_label = exp_dataset.iloc[[2]].sentiment.item(),
    predictor = finetuned_model,
    tokenizer = tokenizer, 
    att_ratio = perturb_ratio,
    perturbation_technique = perturbation_technique,
    lang_codemix = perturb_lang,
    sim_predictor = use
)

ic| len_text: 5
ic| end_time_logit: 0.0158
ic| leave_1_probs: tensor([[5.8145e-04, 1.2208e-03, 9.9820e-01],
                           [3.1326e-03, 9.9333e-01, 3.5423e-03],
                           [6.4712e-04, 1.3099e-03, 9.9804e-01],
                           [5.1722e-04, 1.5441e-03, 9.9794e-01],
                           [9.0873e-04, 1.1874e-03, 9.9790e-01]], device='cuda:0')
ic| orig_prob_extended: array([0.99799299, 0.99799299, 0.99799299, 0.99799299, 0.99799299])
ic| end_time_arrops: 0.2159
ic| running_time_a: 0.3671
ic| end_time_pert: 0.2674
ic| end_time_use: 0.0
ic| first_perturbation_sim_score: 1.0
ic| running_time: 0.8207


('mau marah sama koneksi Indosat',
 1.0,
 2,
 array([5.00e-04, 1.50e-03, 9.98e-01]),
 2,
 array([5.00e-04, 1.50e-03, 9.98e-01]),
 {'indosat': 'Indosat'},
 0.8207)

### Coba optimize di USE nya

In [ ]:
attack(
    text_ls = exp_dataset.iloc[[2]].text.item(),
    true_label = exp_dataset.iloc[[2]].sentiment.item(),
    predictor = finetuned_model,
    tokenizer = tokenizer, 
    att_ratio = perturb_ratio,
    perturbation_technique = perturbation_technique,
    lang_codemix = perturb_lang,
    sim_predictor = use
)

### Coba Optimize step 2+codemix perturbation pake batch

In [ ]:
attack(
    text_ls = exp_dataset.iloc[[48]].text.item(),
    true_label = 0,
    predictor = finetuned_model,
    tokenizer = tokenizer, 
    att_ratio = perturb_ratio,
    perturbation_technique = perturbation_technique,
    lang_codemix = perturb_lang,
    sim_predictor = use
)

In [ ]:
def codemix_perturbation(words, target_lang, words_perturb):
    """
    'su': 'sundanese'
    'jw': 'javanese'
    'ms': 'malay'
    'en': 'english'
    """
    translator = GoogleTranslator(source="id", target=target_lang)
    new_wp = []
    for wp in words_perturb:
        if wp[1].isalpha():
            new_wp.append(wp[1])
    
    if len(new_wp) == 1:
        new_wp_trans = dict(zip(new_wp, translator.translate(new_wp[0])))
    elif len(new_wp) == 0:
        return ' '.join(new_words), {}
        
    new_wp_trans = dict(zip(new_wp, translator.translate_batch(new_wp)))
    
    supported_langs = ["su", "jw", "ms", "en", "fr", "it"]
    
    if target_lang not in supported_langs:
        raise ValueError('Language Unavailable')
    
    new_words = words.copy()
    
    if len(words_perturb) >= 1:
        for perturb_word in new_wp_trans.keys():
            new_words = [new_wp_trans[perturb_word] if word == perturb_word and word.isalpha() else word for word in new_words]

    return ' '.join(new_words), new_wp_trans

def synonym_replacement(words, words_perturb):    
    # new_words = words.copy()
    new_wp = []
    for wp in words_perturb:
        if wp[1].isalpha():
            new_wp.append(wp[1])
    
    if len(new_wp) == 1:
        new_wp_trans = dict(zip(new_wp, get_synonyms(new_wp[0])[0]))
    elif len(new_wp) == 0:
        return ' '.join(new_words), {}
    
    wp_trans=[]
    for wp in new_wp:
        wp_trans.append(get_synonyms(wp)[0])
        print(wp,wp_trans)
    
    new_wp_trans = dict(zip(new_wp, wp_trans))
    
    new_words = words.copy()
    
    if len(words_perturb) >= 1:
        for perturb_word in new_wp_trans.keys():
            new_words = [new_wp_trans[perturb_word[1]] if word == perturb_word[1] and word.isalpha() else word for word in new_words]

    sentence = ' '.join(new_words)
    
    return sentence, new_wp_trans

In [ ]:
word = 'buat kalian yang cuma tahu tempat makan malam romantis yang ada di dago , boleh dicoba deh makan di burgundy sini , di sini sih menurut saya tempat yang enak banget buat ngadem selain tempat nya yang hening kita juga bisa melihat rerumputan yang hijau meski cuma di taman nya , hehehe . tapi balik lagi buat saya sih makanan , haha . tidak usah diragukan .'
words = word.split()
wp = [(59, 'diragukan', 0.001480978060399707), (27, 'enak', 0.0006103660272316347), (18, 'burgundy', 0.0005351185233486433), (55, 'makanan', 0.000532135112141674), (12, 'dago', 0.0004499789485095107), (8, 'romantis', 0.0003965279976370084), (46, 'taman', 0.00039581217539819136), (40, 'rerumputan', 0.00039318750260974866), (7, 'malam', 0.000384060906586825)]
target_lang = "en"

start_time = time.time()
print(synonym_replacement(words, wp))
print(round(time.time() - start_time, 2))

In [ ]:
mbert-sentiment-codemixing-jw-adv-0.6-valid
# 279/1260
850/1260
new_wp_trans = dict(zip(new_wp, translator.translate(new_wp[0])))
IndexError: list index out of range

In [ ]:
attack(
    text_ls = exp_dataset.iloc[[5]].text.item(),
    true_label = exp_dataset.iloc[[5]].sentiment.item(),
    predictor = finetuned_model,
    tokenizer = tokenizer, 
    att_ratio = 0.6,
    perturbation_technique = perturbation_technique,
    lang_codemix = "jw",
    sim_predictor = use
)

In [ ]:
exp_dataset.iloc[[7]]

In [ ]:

import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))
# !pip list cudnn

In [ ]:
texts = ['diragukan', 'enak', 'burgundy', 'makanan', 'dago', 'romantis', 'taman', 'rerumputan', 'malam']

start_time = time.time()
translated_1 = []
translator = GoogleTranslator(source="id", target="en")
for txt in texts:
    translated_1.append(translator.translate(txt))
print(round(time.time() - start_time, 2))

In [ ]:
start_time = time.time()
translated_2 = translator.translate_batch(texts)
print(round(time.time() - start_time, 2))

In [ ]:
# !. myenv/bin/activate
# !pip uninstall tensorflow -y
# !pip uninstall tensorflow-gpu -y

!pip install tensorflow --ignore-installed

In [83]:
import transformers
print(transformers.__version__)

4.17.0


In [27]:
def logit_prob(text_ls, predictor, tokenizer):
    # original_text = text_ls
    # print(text_ls)
    subwords = tokenizer.encode(text_ls)
    subwords = torch.LongTensor(subwords).view(1, -1).to(predictor.device)

    logits = predictor(subwords)[0]
    orig_label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
    
    orig_probs = F.softmax(logits, dim=-1).squeeze()
    orig_prob = F.softmax(logits, dim=-1).squeeze()[orig_label].detach().cpu().numpy()
    
    return orig_label, orig_probs, orig_prob

In [ ]:
# %export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/lib64
!echo $LD_LIBRARY_PATH

In [33]:
def logit_prob(text_ls, predictor, tokenizer):
    with torch.inference_mode():
        subwords = np.array(tokenizer(text_ls)['input_ids'])
        # ic(subwords)
        if subwords.ndim <= 1:
            subwords = torch.tensor(subwords).view(1, -1).to(predictor.device)
            logits = predictor(subwords)[0]
            orig_label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
            orig_probs = F.softmax(logits, dim=-1).squeeze()
            orig_prob = F.softmax(logits, dim=-1).squeeze()[orig_label].detach().cpu().numpy()
        else:
            subwords = torch.tensor(subwords).to(predictor.device)
            logits = finetuned_model(subwords)[0]
            orig_label = [t.squeeze().item() for t in torch.topk(logits, k=1, dim=-1)[1]]
            orig_probs = F.softmax(logits, dim=-1).squeeze()
            orig_prob = [F.softmax(logits, dim=-1).squeeze()[idx][label].detach().cpu().numpy() for idx, label in enumerate(orig_label)]
    
    return orig_label, orig_probs, orig_prob

In [ ]:
subwords = np.array([
    [2, 4, 4700, 500, 5622, 7884, 3],
    [2, 422, 4, 500, 5622, 7884, 3],
    [2, 422, 4700, 4, 5622, 7884, 3],
    [2, 422, 4700, 500, 4, 7884, 3],
    [2, 422, 4700, 500, 5622, 4, 3]
])
subwords = torch.tensor(subwords).to(finetuned_model.device)
subwords

tensor([[   2,    4, 4700,  500, 5622, 7884,    3],
        [   2,  422,    4,  500, 5622, 7884,    3],
        [   2,  422, 4700,    4, 5622, 7884,    3],
        [   2,  422, 4700,  500,    4, 7884,    3],
        [   2,  422, 4700,  500, 5622,    4,    3]], device='cuda:0')

In [15]:
subwords = np.array([2, 4, 4700, 500, 5622, 7884, 3])
print(subwords.ndim)
subwords = torch.tensor(subwords).view(1,-1).to(finetuned_model.device)

1


In [17]:
subwords

tensor([[   2,    4, 4700,  500, 5622, 7884,    3]], device='cuda:0')

In [79]:
start_time_use = time.time()
subwords = np.array([
    [2, 4, 4700, 500, 5622, 7884, 3],
    [2, 422, 4, 500, 5622, 7884, 3],
    [2, 422, 4700, 4, 5622, 7884, 3],
    [2, 422, 4700, 500, 4, 7884, 3],
    [2, 422, 4700, 500, 5622, 4, 3]
])
subwords = torch.tensor(subwords).to(finetuned_model.device)
ic(subwords)
logits = finetuned_model(subwords)[0]
ic(logits)
orig_label = [t.squeeze().item() for t in torch.topk(logits, k=1, dim=-1)[1]]
ic(orig_label)
orig_probs = F.softmax(logits, dim=-1).squeeze()
ic(orig_probs)
orig_prob = [F.softmax(logits, dim=-1).squeeze()[idx][label].detach().cpu().numpy() for idx, label in enumerate(orig_label)]
ic(orig_prob)
end_time_use = round(time.time() - start_time_use, 4)
ic(end_time_use)

ic| subwords: tensor([[   2,    4, 4700,  500, 5622, 7884,    3],
                      [   2,  422,    4,  500, 5622, 7884,    3],
                      [   2,  422, 4700,    4, 5622, 7884,    3],
                      [   2,  422, 4700,  500,    4, 7884,    3],
                      [   2,  422, 4700,  500, 5622,    4,    3]], device='cuda:0')
ic| logits: tensor([[-2.7573, -2.0155,  4.6909],
                    [-2.1742,  3.5850, -2.0513],
                    [-2.7181, -2.0129,  4.6229],
                    [-2.9572, -1.8635,  4.6078],
                    [-2.4672, -2.1997,  4.5342]], device='cuda:0',
                   grad_fn=<AddmmBackward0>)
ic| orig_label: [2, 1, 2, 2, 2]
ic| orig_probs: tensor([[5.8145e-04, 1.2208e-03, 9.9820e-01],
                        [3.1326e-03, 9.9333e-01, 3.5423e-03],
                        [6.4712e-04, 1.3099e-03, 9.9804e-01],
                        [5.1722e-04, 1.5441e-03, 9.9794e-01],
                        [9.0873e-04, 1.1874e-03, 9.9790e-01]], d

0.5198

In [72]:
# orig_label
# orig_probs = F.softmax(logits, dim=-1).squeeze()
# orig_probs
for idx, label in enumerate(orig_label):
    print(label)
    print(F.softmax(logits, dim=-1).squeeze()[idx][label].detach().cpu().numpy())
orig_prob = [F.softmax(logits, dim=-1).squeeze()[idx][label].detach().cpu().numpy() for idx, label in enumerate(orig_label)]
# orig_prob = F.softmax(logits, dim=-1).squeeze()
print(orig_prob)

2
0.99819773
1
0.9933252
2
0.99804294
2
0.9979386
2
0.9979038
[array(0.99819773, dtype=float32), array(0.9933252, dtype=float32), array(0.99804294, dtype=float32), array(0.9979386, dtype=float32), array(0.9979038, dtype=float32)]
